## Installation of required dependencies

In [1]:
!pip install langchain
!pip install langchain-ollama
!pip install python-dotenv
!pip install ipython


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


##Message History

In [2]:
from dotenv import load_dotenv
import os
#load langsmith connectivity environment variables
dotenv=load_dotenv('.env')
os.getenv('LANGSMITH_ENDPOINT')

'https://api.smith.langchain.com'

In [3]:
from langchain_ollama import ChatOllama
chat_with_llama32=ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3",
    temperature=0.9
)

## Create a chain of runnable

In [ ]:
import random
from langchain_core.prompts import  ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

#session store and get message history
session_store={}
def get_session_message_history(sessionID):
    return SQLChatMessageHistory(session_id=sessionID, connection_string="sqlite:///chathistory.db")

#create chat prompt
chat_prompt_template = ChatPromptTemplate([
    ('placeholder', "{history}"),
    ('human', '{new_message}')
])

#create chain for the chat session
chain = chat_prompt_template | chat_with_llama32 | StrOutputParser()

#create the runnable to manage message history for the chain
runnable_with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history=get_session_message_history,
    input_messages_key="new_message",
    history_messages_key="history"
)
runnable_with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_message_history at 0x10b88bc40>, input_messages_key='new_message', history_messages_key='history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [6]:
#create a session id
random.seed(10)
session_id=str(random.randint(1,99999))

#get_session_message_history(session_id).clear()

In [7]:
#begin the chat
response1 = runnable_with_message_history.invoke(
    {"new_message":"What is the benefit of running LLM in local machine?"},
    config={"configurable":{"session_id": session_id}}
)
print(response1)

ArgumentError: Could not parse SQLAlchemy URL from given URL string

In [84]:
#new prompt on the chat
response2 = runnable_with_message_history.invoke(
    {"new_message":"How about cloud?"},
    config={"configurable":{"session_id": session_id}}
)
print(response2)

<think>
Okay, the user asked about the benefits of running LLMs on local machines, and now they're asking "How about cloud?" So they want to know the advantages of using the cloud instead. Let me start by recalling the previous answer where I listed the pros of local deployment.

I need to structure the cloud benefits similarly but make sure to contrast where appropriate. Let me think about the main points. Cloud computing offers scalability, cost efficiency through pay-as-you-go, access to advanced infrastructure, and maybe better support for training large models. Also, collaboration and integration with other services could be a point.

Wait, the user might be comparing the two approaches. They might be deciding between on-premise and cloud. So I should highlight the pros of cloud while acknowledging that it's not always the best choice. Maybe mention scenarios where cloud is better, like when you need to train models, handle variable workloads, or don't have the hardware.

I should